In [74]:
from pandas import DataFrame
import random

In [143]:
random.gauss(0, 0.5)

-0.10340764215680279

In [170]:
paramsToStore = {"Time": [0,1,2,3,4,5,0,1,2,3],
                "TTC": [random.uniform(0, 10) for _ in range(10)],
                "DTO": [random.uniform(0, 20) for _ in range(10)],
                "JERK": [abs(random.gauss(0, 20)) for _ in range(10)],
                "Speed": [random.uniform(0, 15) for _ in range(10)],
                "AngularSpeedX": [random.gauss(0, 0.15) for _ in range(10)],
                "AngularSpeedY": [random.gauss(0, 0.15) for _ in range(10)],
                "AngularSpeedZ": [random.gauss(0, 0.1) for _ in range(10)],
                "COL": [0,0,0,1,0,0,0,0,1,0]}
df = DataFrame(paramsToStore)
# df.set_index("Time", inplace=True)
# df.to_csv(PATH+"/data/testGenerateData.csv", index=False)
df.head(10)

,Time,TTC,DTO,JERK,Speed,AngularSpeedX,AngularSpeedY,AngularSpeedZ,COL
0,0,9.926618,9.225689,16.527835,11.913120,0.245439,-0.038028,-0.028775,0
1,1,4.326807,14.759939,31.684829,8.228288,-0.239379,-0.075622,0.217454,0
2,2,9.465413,7.914834,13.927447,7.389057,0.118709,0.064934,0.156433,0
3,3,7.150190,7.612916,26.800420,8.677762,-0.070440,0.036182,-0.174801,1
4,4,8.327346,13.877329,6.943052,6.008791,-0.012583,0.121818,0.026721,0
5,5,5.400700,11.423889,16.957657,5.183449,-0.113824,0.097789,-0.056989,0
6,0,1.604582,19.046982,13.059100,2.571828,-0.035204,-0.115572,0.106931,0
7,1,5.461979,9.752484,24.657093,6.807153,-0.004729,-0.168147,-0.041058,0
8,2,1.771344,17.642109,8.192372,13.583729,0.149126,-0.012271,0.111603,1
9,3,3.555797,9.641721,2.730820,3.327950,0.250741,0.054078,-0.138034,0


In [187]:
def makeDataUsable(df, pastImportance=3):
    ttcNames = [f"TTC{i}" for i in range(1, pastImportance+1)]
    dtoNames = [f"DTO{i}" for i in range(1, pastImportance+1)]
    jerkNames = [f"JERK{i}" for i in range(1, pastImportance+1)]
    speedNames = [f"Speed{i}" for i in range(1, pastImportance+1)]
    columns=ttcNames+dtoNames+jerkNames+speedNames+["COL"]
    dataDict = {col: [] for col in columns}

    for i, row in df.iterrows():
        print(f"{i}, time: {int(row['Time'])}")
        if row["Time"] < pastImportance-1:
            print("\tNeed more data")
            continue
        print("\t+++++Do Stuff")


    return DataFrame(dataDict)

makeDataUsable(df)

0, time: 0
	Need more data
1, time: 1
	Need more data
2, time: 2
	+++++Do Stuff
3, time: 3
	+++++Do Stuff
4, time: 4
	+++++Do Stuff
5, time: 5
	+++++Do Stuff
6, time: 0
	Need more data
7, time: 1
	Need more data
8, time: 2
	+++++Do Stuff
9, time: 3
	+++++Do Stuff


,TTC1,TTC2,TTC3,DTO1,DTO2,DTO3,JERK1,JERK2,JERK3,Speed1,Speed2,Speed3,COL
